2024-09-19 - Ronen Shekel

In [1]:
import matplotlib
matplotlib.use('TKAgg')
%load_ext autoreload
%autoreload 2
import os 
import datetime
import glob 
import numpy as np
import matplotlib.pyplot as plt
from pianoq.simulations.mplc_sim.mplc_sim import MPLCSim
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.singles_scan import signal_scan, idler_scan, get_signal_scanner, get_idler_scanner
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots, get_imaging_masks
from pianoq_results.scan_result import ScanResult
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCSimResult
from pianoq.lab.mplc.mplc_device import MPLCDevice
from pianoq.lab.photon_scan import PhotonScanner
from pianoq.misc.misc import run_in_thread, run_in_thread_simple
from pianoq.misc.mplt import mimshow, mplot
from pianoq.simulations.mplc_sim.create_wfm_masks import create_WFM_diffuser_masks
from pianoq.lab.mplc.discrete_photon_scanner import DiscretePhotonScanner
import time 
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.find_discreet_phases import PhaseFinder

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_19_filters_80nm'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])
U_no = 1
phases_no = 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# HW

In [2]:
# Motors
backlash = 0
wait_after_move = 0.3
from pianoq.lab.mplc.consts import thorlabs_x_serial, thorlabs_y_serial
from pianoq.lab.thorlabs_motor import ThorlabsKcubeDC, ThorlabsKcubeStepper
from pianoq.lab.zaber_motor import ZaberMotors
zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
mxs = zaber_ms.motors[1]
mys = zaber_ms.motors[0]
print('Got Zaber motors')

mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
print('Got Thorlabs motors')

# MPLC
mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')

# Timetagger
from pianoq.lab.time_tagger import QPTimeTagger
from pianoq.lab.mplc.consts import TIMETAGGER_DELAYS, TIMETAGGER_COIN_WINDOW
tt = QPTimeTagger(integration_time=1, remote=True,
                  single_channel_delays=TIMETAGGER_DELAYS, coin_window=TIMETAGGER_COIN_WINDOW)
print('Got Time tagger')

Got Zaber motors
Got Thorlabs motors
Got MPLC
Got Time tagger


# Single counts 

In [4]:
resolution = 1 
filter_type = '20nm'
integration_time = 1 
tt.set_integration_time(integration_time)
scanner_sig, _, _, _ = get_signal_scanner(integration_time=integration_time, coin_window=2e-9, resolution=resolution, half_scan=True, no_hw_mode=True)
scanner_idl, _, _, _ = get_idler_scanner(integration_time=integration_time, coin_window=2e-9, resolution=resolution, half_scan=True, no_hw_mode=True)

masks_path = glob.glob(rf'{dir_path}\U{U_no}U*.masks')[0]

msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
mplc.load_masks(masks, linear_tilts=True)
mplc.restore_location()

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
singles_idler_path = fr'{dir_path}\{timestamp}_idler_res_{resolution}_filter_{filter_type}.scan'
singles_signal_path = fr'{dir_path}\{timestamp}_signal_res_{resolution}_filter_{filter_type}.scan'

scanner_idl.saveto_path = singles_idler_path
scanner_sig.saveto_path = singles_signal_path
scanner_idl.scan(x_motor=mxi, y_motor=myi, ph=tt)    
scanner_sig.scan(x_motor=mxs, y_motor=mys, ph=tt)

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 1404. Singles2: 2084. Coincidence: 0.00.
dur: 2. pix: 0, 1. Singles1: 1280. Singles2: 2025. Coincidence: 0.00.
dur: 4. pix: 0, 2. Singles1: 1392. Singles2: 1984. Coincidence: 0.00.
dur: 6. pix: 0, 3. Singles1: 1404. Singles2: 1979. Coincidence: 0.00.
dur: 8. pix: 0, 4. Singles1: 1371. Singles2: 2026. Coincidence: 0.00.
dur: 9. pix: 0, 5. Singles1: 1482. Singles2: 2083. Coincidence: 0.00.
dur: 11. pix: 0, 6. Singles1: 1507. Singles2: 2059. Coincidence: 0.00.
dur: 13. pix: 0, 7. Singles1: 1436. Singles2: 1992. Coincidence: 0.00.
dur: 15. pix: 0, 8. Singles1: 1423. Singles2: 1969. Coincidence: 0.00.
dur: 17. pix: 0, 9. Singles1: 1367. Singles2: 2073. Coincidence: 0.00.
dur: 19. pix: 0, 10. Singles1: 1447. Singles2: 2046. Coincidence: 0.00.
dur: 21. pix: 0, 11. Singles1: 1472. Singles2: 2046. Coincidence: 0.00.
dur: 22. pix: 0, 12. Singles1: 1491. Singles2: 2032. Coincidence: 0.00.
dur: 24. pix: 0, 13. Singles1: 1402

(array([[1956., 2009., 2026., ..., 1935., 1986., 1955.],
        [1975., 2029., 1930., ..., 2051., 2001., 1994.],
        [1994., 2001., 2014., ..., 1936., 1944., 1967.],
        ...,
        [1952., 1949., 1981., ..., 1976., 2048., 1982.],
        [2083., 1952., 1929., ..., 1827., 1892., 1887.],
        [1869., 1862., 1822., ..., 1772., 1891., 1794.]]),
 array([[1886., 2021., 1975., ..., 2136., 1931., 1894.],
        [1861., 1874., 1970., ..., 2136., 1951., 1859.],
        [1933., 1919., 2050., ..., 2139., 1951., 1750.],
        ...,
        [1543., 1519., 1531., ..., 2026., 1965., 1951.],
        [1575., 1508., 1571., ..., 1851., 1812., 1796.],
        [1374., 1409., 1536., ..., 1889., 1740., 1799.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]))

# correlations

# mark spots locations 80nm
Very similar to 20nm, but why not. 

In [11]:
path_sig = glob.glob(fr'{dir_path}\*signal_res_1_filter_80*.scan')[0]
path_idl = glob.glob(fr'{dir_path}\*idler_res_1_filter_80*.scan')[0]

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
locs_signal_path = fr'{dir_path}\{timestamp}_s2_sig_filter_80nm.locs'
locs_idler_path = fr'{dir_path}\{timestamp}_s1_idl_filter_80nm.locs'

res_sig = ScanResult()
res_idl = ScanResult()
res_idl.loadfrom(path_idl)
res_sig.loadfrom(path_sig)
# res_idl.show_singles()
# res_sig.show_singles()

res_sig.get_xys(2, saveto_path=locs_signal_path)
res_idl.get_xys(1, saveto_path=locs_idler_path)

Saved to G:\My Drive\Projects\MPLC\results\lab\2024_09_19_filters_80nm\2024_09_19_14_00_14_s1_idl_filter_80nm.locs


[(9.115608558108558, 3.100520313020313),
 (9.049741092241092, 2.7253621378621378),
 (9.023966866466868, 2.3702505827505824),
 (8.998192640692642, 2.000820013320013),
 (8.975282217782219, 1.6256618381618382)]

# Phases

In [17]:
# masks 
masks_path = glob.glob(rf'{dir_path}\U{U_no}U*.masks')[0]
msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
mplc.load_masks(masks, linear_tilts=True)

from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
r1 = PhaseFinderResult()
phase_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_19_filters_80nm\2024_09_19_14_02_19_80nm.phases"
r1.loadfrom(phase_path)

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
phases_path = fr'{dir_path}\{timestamp}_filter_80nm.phases'
integration_time = 10
pf = PhaseFinder(mplc=mplc, integration_time=integration_time, remote_tagger=True, saveto_path=phases_path,
                 modes_to_keep=modes_to_keep, intial_phases=r1.phases, coin_window=1e-9, no_hw_mode=True)

pf.zaber_ms = zaber_ms
pf.m_sig_x = mxs
pf.m_sig_y = mys
pf.m_idl_x = mxi
pf.m_idl_y = myi
pf.time_tagger = tt
pf.time_tagger.set_integration_time(integration_time)

# locs 
locs_idl_path = glob.glob(fr'{dir_path}\*idl*filter_80nm*.locs')[0]
locs_sig_path = glob.glob(fr'{dir_path}\*sig*filter_80nm*.locs')[0]
locs_idl = np.load(locs_idl_path)['locs']
locs_sig = np.load(locs_sig_path)['locs']

# i,j of corr matrix that is supposed to be strong 
i = 2
j = 2
pf.m_idl_x.move_absolute(locs_idl[i, 0])
pf.m_idl_y.move_absolute(locs_idl[i, 1])
pf.m_sig_x.move_absolute(locs_sig[j, 0])
pf.m_sig_y.move_absolute(locs_sig[j, 1])        
mplc.restore_location()
time.sleep(1)
pf.find_phases()

0,0
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9
1,0
1,1
1,2
1,3
1,4
1,5
1,6
1,7
1,8
1,9
2,0
2,1
2,2
2,3
2,4
2,5
2,6
2,7
2,8
2,9
3,0
3,1
3,2
3,3
3,4
3,5
3,6
3,7
3,8
3,9
4,0
4,1
4,2
4,3
4,4
4,5
4,6
4,7
4,8
4,9
5,0
5,1
5,2
5,3
5,4
5,5
5,6
5,7
5,8
5,9
6,0
6,1
6,2
6,3
6,4
6,5
6,6
6,7
6,8
6,9
7,0
7,1
7,2
7,3
7,4
7,5
7,6
7,7
7,8
7,9
8,0
8,1
8,2
8,3
8,4
8,5
8,6
8,7
8,8
8,9
9,0
9,1
9,2
9,3
9,4
9,5
9,6
9,7
9,8
9,9


# Diffuser

In [4]:
masks_path = glob.glob(rf'{dir_path}\*diffuser_d1d*.masks')[0]
msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
mplc.load_masks(masks, linear_tilts=True)

# Put idler in OK place 
mxi.move_absolute(8.95)
myi.move_absolute(2.1)

# scan signal
start_x = 11
end_x = 12.1
start_y = 8.7
end_y = 10.3
integration_time = 15
tt.set_integration_time(integration_time)

x_pixels = 22
y_pixels = 32
pixel_size_x = 0.05
pixel_size_y = 0.05


timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
path = f'{dir_path}\\{timestamp}_two_ph_speckles_d1d_exp_{integration_time}s_filter_80nm.scan'
scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, 
                        is_timetagger=True, coin_window=2e-9, saveto_path=path)

single1s, single2s, coincidences = scanner.scan(ph=tt , x_motor=mxs, y_motor=mys)

Moving to starting position...
starting scan
dur: 0. pix: 0, 0. Singles1: 15818. Singles2: 1820. Coincidence: 0.00.
dur: 15. pix: 0, 1. Singles1: 16002. Singles2: 2052. Coincidence: 0.07.
dur: 31. pix: 0, 2. Singles1: 16000. Singles2: 2329. Coincidence: 0.20.
dur: 47. pix: 0, 3. Singles1: 15930. Singles2: 2565. Coincidence: 0.13.
dur: 62. pix: 0, 4. Singles1: 16036. Singles2: 2760. Coincidence: 0.47.
dur: 78. pix: 0, 5. Singles1: 16030. Singles2: 2876. Coincidence: 0.13.
dur: 93. pix: 0, 6. Singles1: 16088. Singles2: 3007. Coincidence: 0.20.
dur: 109. pix: 0, 7. Singles1: 16036. Singles2: 3157. Coincidence: 0.27.
dur: 124. pix: 0, 8. Singles1: 16117. Singles2: 3252. Coincidence: 0.33.
dur: 140. pix: 0, 9. Singles1: 16107. Singles2: 3386. Coincidence: 0.13.
dur: 156. pix: 0, 10. Singles1: 16088. Singles2: 3365. Coincidence: 0.07.
dur: 171. pix: 0, 11. Singles1: 16152. Singles2: 3272. Coincidence: 0.33.
dur: 187. pix: 0, 12. Singles1: 16070. Singles2: 3172. Coincidence: 0.20.
dur: 202. p